In [ ]:
#This program would take images from a local folder, read them and identify there faces, it would first crop out the image
#itself and then place facial feature markers on the crop images. After the landmarks are place, then it is calculated
#from multiple points towards the center or the dimension of the one feature. It would then all be returns in a dataframe
#and all images that either doesn't get cropped or the face isn't able to be marked, the image would be skip through. Although
#the images that has been sucsessfully crop doesn't necessarily means that the facial features would be sucsessfully identified.

#Start Timer (Run Time - )
import time
start = time.time()

import cv2
import numpy as np
import dlib
import array
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# This Method would first import the model that we would be using, then create a duplicate image of the original so it
# would not alter the original. Onwards it would convert from RBG towards Gray Scale and using the haarcascade, it would
# crop the image and create a image with just the face. The crop image would be saved towards the path and returned.

def detect_faces(test_image, scaleFactor = 1.3):
    
    #Import the cascadeClassifier XML files 
    cascade = cv2.CascadeClassifier('/Users/kp/Desktop/Generali/Facial Landmark/list/haarcascade_frontalface_default.xml')

    #Create a backup image that can be modified, turn the image copy to gray scale and use the detectMultiscale function
    #to use the parameters grayscale image, scale factor, min neighbor and minimum size. DetectMultiScale detects objects 
    #of different sizes in the input image. The detected objects are returned as a list of rectangles.
    image_copy = test_image.copy()
    gray_image = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
    faces_rect = cascade.detectMultiScale(gray_image, scaleFactor=scaleFactor, minNeighbors=3, minSize=(10,10))
    
    #For loop which calls the rectangle funtion for the crop box, color and then write it to the specific path in the 
    #computer.
    for (x, y, w, h) in faces_rect:
        a = cv2.rectangle(image_copy, (x, y), (x+w, y+h), (0, 255, 0), 1)
        b = roi_color = image_copy[y:y + h, x:x + w] 
        c = cv2.imwrite('/Users/kp/Desktop/Generali/Facial Landmark/test_image/Crop/Crop_' + counter, roi_color)

    #Returns faces_rect for identifying if the image is empty or not, image_copy the array of the image itself.
    return faces_rect, image_copy

In [ ]:
#This method would take the image that detect_face returns and use it as an array. By importing the model shape predictor
#this would allow this method to produce markers on the image. It would also convert the image from RBG to gray simlarly
#towards the previous method. After the image and landmark coordinate has been placed, it would be all stored in one list
#and returned for the next list to use.

def landmark_placer(test_image):
    
    #Declaration of the array, create a duplicate of image to modify
    fulllist = []
    image_copy = test_image.copy()
    
    #Calls dlib and use the get frontal face detector method and use the shape_predictor file for detecting the facial 
    #landmarks and plot it. Conversion of image towards gray scale and storing it in the variable faces
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/Users/kp/Desktop/Generali/Facial Landmark/dlib project/shape_predictor_68_face_landmarks.dat')
    gray = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)   

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        
    #Placing the landmarks on the face, the loop would run 68 times because there is a fix amount of points that are available
    #and would be plot on the image, the fulllist would store the coordinates of the points with a index counter. This will keep 
    #on going unless there is a fail
    try:
        landmarks = predictor(gray, face)
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(image_copy, (x, y), 1, (0, 255, 1), 1)
            fulllist.append([n+1,x,y])
        
    #During a case of a failed/empty array, it will pass and skip that image
    except UnboundLocalError as error:
        pass
    
    #Return faces which is the identifier if the image is a failed one and fulllist which stores the coordinates.
    return faces, fulllist

In [ ]:
#This method distance_cal main function is from each end of each landmarks. It uses what is returned by landmark placer
#it would calculate the distance of it which in example for the eye, it would calculate the diameter of the left eye 
#and return that in a dataframe format. 

def distance_cal (fulllist):
    
    #Declaration of the list and storing fulllist from the other method towards a new list
    original = fulllist
    distance_list = []
    temp_list = []
    calculated_list = []
    
    #From the original list, select the coordinates that is wanted and keeping it in the distance 
    #list seperate from the original list. The points are for calculating the distance from end to 
    #end of the left eye, it would do the same for each landmark
    for n in range(0,len(original)):
        if n == 0 or n == 16 or n == 17 or n ==21 or n == 22 or n == 26 or n == 27 or n == 30 or n == 31 or n == 35 or n == 36 or n == 39 or n == 42 or n == 45 or n == 48 or n == 54:
            x = original[n][1]
            y = original[n][2]
            distance_list.append([x,y]) 
     
    #The specific points that has been collected would then be calculated towards the distance formula using the Euclidean Distance
    #and storing it towards the temp list
    for i in range(0,16,2):
        dis = np.sqrt(np.square(distance_list[i+1][0] - distance_list[i][0]) + np.square(distance_list[i+1][1] - distance_list[i][1]))
        temp_list.append(dis)
        
    #After collection of the distance, it would then be appended towards a new list which would then be added towards the database,
    #then with the database, the header column labeling what each column represent would be printed. Return the dataframe
    calculated_list.append (temp_list)
    df = pd.DataFrame(calculated_list)
    df.columns = ['Chin', 'Right Eyebrow', 'Left Eyebrow', 'Long Nose', 'Bottom Nose', 'Right Eyes', 'Left Eyes', 'Mouth']
    return df

In [ ]:
#This method point_cal would take what is returned by landmark_placer and calculate each distance from one point to another.
#Some landmark like nose and eye would be measured from the center point and would be compared towards the distance 
#between the multiple points on the mouth, eyebrow and chin(cheeks and chin). From there distance would be calculated
#and returned in the same dataframe method.

def point_cal (fulllist):
    
    #Declaration of the list and storing the imported fulllist and storing it in a new list called ori
    ori = fulllist
    chin = []
    rightbrow = []
    leftbrow = []
    upperlips = []
    lowerlips = []
    completelist = []
    lefteye = [[counter, "Name"]]
    righteye = []
    nose = []
    indexcount = 0

    #For loop to seperate the coordinates required to compare the center points with, this would take out the coordinates
    #for chin, eye brows and mouth. Combining all the individual points being compared into one list
    for n in range(0,68):
        if n >= 0 and n <= 16:
            x = ori[n][1]
            y = ori[n][2]
            chin.append([x,y])
        elif n >= 17 and n <=21:
            x = ori[n][1]
            y = ori[n][2]
            rightbrow.append([x,y])
        elif n >= 22 and n <= 26:
            x = ori[n][1]
            y = ori[n][2]
            leftbrow.append([x,y])
        elif n >= 48 and n <=54 or n >= 60 and n <= 64:
            x = ori[n][1]
            y = ori[n][2]
            upperlips.append([x,y])
        elif n >= 55 and n <= 59 or n >= 65 and n <=67:
            x = ori[n][1]
            y = ori[n][2]
            lowerlips.append([x,y])
    completelist = [chin, rightbrow, leftbrow, upperlips, lowerlips] 
    
    #Calculating the mid point for the eyes and nose
    left_center_eye = [(ori[45][1]+ori[42][1])/2, (ori[45][2]+ori[42][2])/2]
    right_center_eye = [(ori[40][1]+ori[37][1])/2, (ori[40][2]+ori[37][2])/2]
    nose_center = [(ori[35][1]+ori[31][1])/2, (ori[35][2]+ori[31][2])/2]
    
    #Nested loop for selecting the points in the 2d list "Completelist", this would go through and compare all the center
    #points of left and right eye, and nose with the points in the completelist and storing it in another new list. At the 
    #end, there is a index count that would go through the if statements
    for attr in completelist:
        for point in attr: 
            if indexcount >= 0 and indexcount <= 16:
                lefteye.append([np.sqrt(np.square(point[0] - left_center_eye[0]) + np.square(point[1] - left_center_eye[1])), 'Left Eye to Chin Point: ' + str(indexcount + 1)])
                righteye.append([np.sqrt(np.square(point[0] - right_center_eye[0]) + np.square(point[1] - right_center_eye[1])), 'Right Eye to Chin Point: ' + str(indexcount + 1)])
                nose.append([np.sqrt(np.square(point[0] - nose_center[0]) + np.square(point[1] - nose_center[1])), 'Nose to Chin Point: ' + str(indexcount + 1)])
            elif indexcount >= 17 and indexcount <=21:
                lefteye.append([np.sqrt(np.square(point[0] - left_center_eye[0]) + np.square(point[1] - left_center_eye[1])), 'Left Eye to Right Brow Point: ' + str(indexcount + 1)])
                righteye.append([np.sqrt(np.square(point[0] - right_center_eye[0]) + np.square(point[1] - right_center_eye[1])), 'Right Eye to Right Brow Point: ' + str(indexcount + 1)])
                nose.append([np.sqrt(np.square(point[0] - nose_center[0]) + np.square(point[1] - nose_center[1])), 'Nose to Right Brow Point: ' + str(indexcount + 1)])
            elif indexcount >= 22 and indexcount <= 26:
                lefteye.append([np.sqrt(np.square(point[0] - left_center_eye[0]) + np.square(point[1] - left_center_eye[1])), 'Left Eye to Left Brow Point: ' + str(indexcount + 1)])
                righteye.append([np.sqrt(np.square(point[0] - right_center_eye[0]) + np.square(point[1] - right_center_eye[1])), 'Right Eye to Left Brow Point: ' + str(indexcount + 1)])
                nose.append([np.sqrt(np.square(point[0] - nose_center[0]) + np.square(point[1] - nose_center[1])), 'Nose to Left Brow Point: ' + str(indexcount + 1)])
            elif indexcount >= 27 and indexcount <=39:
                lefteye.append([np.sqrt(np.square(point[0] - left_center_eye[0]) + np.square(point[1] - left_center_eye[1])), 'Left Eye to Upper Lips Point: ' + str(indexcount + 1)])
                righteye.append([np.sqrt(np.square(point[0] - right_center_eye[0]) + np.square(point[1] - right_center_eye[1])), 'Right Eye to Upper Lips Point: ' + str(indexcount + 1)])
                nose.append([np.sqrt(np.square(point[0] - nose_center[0]) + np.square(point[1] - nose_center[1])), 'Nose to Upper Lips Point: ' + str(indexcount + 1)])
            elif indexcount >= 40 and indexcount <= 47:
                lefteye.append([np.sqrt(np.square(point[0] - left_center_eye[0]) + np.square(point[1] - left_center_eye[1])), 'Left Eye to Lower Lips Point: ' + str(indexcount + 1)])
                righteye.append([np.sqrt(np.square(point[0] - right_center_eye[0]) + np.square(point[1] - right_center_eye[1])), 'Right Eye to Lower Lips Point: ' + str(indexcount + 1)])
                nose.append([np.sqrt(np.square(point[0] - nose_center[0]) + np.square(point[1] - nose_center[1])), 'Nose to Lower Lips Point: ' + str(indexcount + 1)])
            indexcount += 1

    #Adding all the points in left and right eye and nose together and storing it in a dataframe. Returning that dataframe
    total = lefteye + righteye + nose   
    df = pd.DataFrame(total) 
    return df

In [ ]:
#This method consist of calling all the other method available in this program. This would first identify the path of the
#folder where all the images that wants to be read and list them down, skipping hidden files from the list. Onwards it 
#would call all the other function then from the dataframe of distance_cal and point_cal, it would produce a table based
#on the facial features and the distance from the specific landmarks that are plotted and combined them into one dataframe
#at the same time for images that doesn't get read, it would pass the image. It would also store a list of images names
#that didnt work and worked.

#Going through the directory and listing down the name of the images that is located in a specific folder that will be read
#and analyse. Then also declaration of dataframe.
x = os.listdir('/Users/kp/Desktop/Generali/Facial Landmark/test_image/ktp')
x.remove(".DS_Store")
transpose_d = pd.DataFrame([])
df_temp = pd.DataFrame([])
sucsess = []
fail = []
x.sort()

#For loop that would go through all the images in the list
for counter in x:
    #Selecting a specific image that name is located in counter and reading it, then it would be use in detect_faces method
    a = cv2.imread('/Users/kp/Desktop/Generali/Facial Landmark/test_image/ktp/' + counter)
    b = detect_faces(a)
    
#     If statement for if the result of b has a length less than one, it would not go through the rest of main and just skip
#     through. b[0] is the identifier helping us to know if the picture failed or not. (Can't be read)
    if len(b[0]) < 1:
        fail.append(counter)
        continue
        
    #Everything else when B is less than 1
    else:
        #Calling the landmark placer method with the result of b[1] which is the data stored from detected faces
        c = landmark_placer (b[1])
        
        #If the result of c's identifier's length is less than 1, it would skip the rest of the step and move on towards
        #the next image
        if len(c[0]) < 1:
            fail.append(counter)
            continue
        else:
            #Calling the method to calculate the points
            df = distance_cal(c[1])
            d = point_cal (c[1])
            
            #Giving the dataframe a name for each main column, the one from point cal is labeled in that method
            df["Name"] = counter
            df = df[['Name', 'Chin', 'Right Eyebrow', 'Left Eyebrow', 'Long Nose', 'Bottom Nose', 'Right Eyes', 'Left Eyes', 'Mouth']]

            #The transpose method would flip the dataframe, then renaming what was the row with index one making it
            #to the header name and dropping that index line to elimiate repition for naming. 
            transpose_d = d.T
            temp = transpose_d.rename(columns = transpose_d.iloc[1])
            newtemp = temp.drop([1])
            
            #Merging the two dataframes and dropping the index so it would be counted properly starting from 0
            df_merge = pd.merge(df, newtemp, on = 'Name', how = "left")
            df_temp = pd.concat([df_temp, df_merge], ignore_index = True)
            sucsess.append(counter)

#Counter
fail.sort() 
fail.insert(0, ('Fail Count: ' + str(len(fail))))
sucsess.sort()
sucsess.insert(0, ('Sucsess Count: ' + str(len(sucsess))))


#End Timer
end = time.time()
print(end - start)

In [ ]:
fail

In [ ]:
sucsess